In [1]:
import warnings
warnings.filterwarnings('ignore')
import json
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BertTokenizerFast, AlbertForQuestionAnswering
import random
import torch
from datasets import Dataset
from tqdm import tqdm

In [5]:
model_name = "UWB-AIR/Czert-B-base-cased"

### Load sqad from JSON file

In [6]:
with open("sqad.json", "r") as f:
    json_data = json.load(f)
    
#json_data[0]
random.shuffle(json_data)
#json_data = json_data.sample(frac=1).reset_index(drop=True)
train = json_data[:9954] #80%
test = json_data[9954:] #20%

In [7]:
print(len(json_data))
print(len(train), int(round(len(train)/len(json_data)*100)),"%")
print(len(test), int(round(len(test)/len(json_data)*100)),"%")

12443
9954 80 %
2489 20 %


In [8]:
for i in json_data[:10]:
    print(i["question"],i["answers"]["text"])

Jak se nazývá východní část Moldávie? ['Besarábie']
Který někdejší rytířský a panský český rod odvozoval svůj přídomek od města Vlašim? ['Jankovští z Vlašimi']
Kdy vznikla Fakulta informatiky na Masarykově univerzitě? ['1994']
Kdy byl natočen film Skřivánci na niti? ['1969']
V kterém československém filmu byl poprvé použit zvuk? ['Když struny lkají']
Kterým filmem debutoval režisér Ivan Passer? ['Intimní osvětlení']
Jaká je zkratka Evropské kosmické agentury? ['ESA']
Surikaty bydlí i v zoo? ['Surikata je velmi častým chovancem zoo.']
Jaké je druhé nejčetnější české křestní jméno? ['Jan']
Jak se nazývá hrdelní vak ptáků pro uskladnění potravy? ['vole']


In [9]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

In [10]:
df_train.head(5)

,answers,context,id,question,title
0,"{'answer_start': [1451], 'text': ['Besarábie']}","Moldávie (rumunsky Moldova, starším českým ozn...",11355,Jak se nazývá východní část Moldávie?,Jak se nazývá východní část Moldávie?
1,"{'answer_start': [0], 'text': ['Jankovští z Vl...",Jankovští z Vlašimi jsou někdejší rytířský a p...,9786,Který někdejší rytířský a panský český rod odv...,Který někdejší rytířský a panský český rod odv...
2,"{'answer_start': [290], 'text': ['1994']}",Fakulta informatiky Masarykovy univerzity (FI ...,10186,Kdy vznikla Fakulta informatiky na Masarykově ...,Kdy vznikla Fakulta informatiky na Masarykově ...
3,"{'answer_start': [78], 'text': ['1969']}",Skřivánci na niti je český hořce poetický film...,4277,Kdy byl natočen film Skřivánci na niti?,Kdy byl natočen film Skřivánci na niti?
4,"{'answer_start': [1295], 'text': ['Když struny...",Česká kinematografie je souhrnné označení pro ...,4152,V kterém československém filmu byl poprvé použ...,V kterém československém filmu byl poprvé použ...


In [11]:
df_test.head(5)

,answers,context,id,question,title
0,"{'answer_start': [12], 'text': ['levostranný']}",Dřevnice je levostranný přítok řeky Moravy ve ...,12242,Jaký přítok je Dřevnice?,Jaký přítok je Dřevnice?
1,"{'answer_start': [0], 'text': ['Samuel Barclay...",Samuel Barclay Beckett [Bekit] (13. dubna 1906...,2490,Kde se narodil Samuel Beckett?,Kde se narodil Samuel Beckett?
2,"{'answer_start': [216], 'text': ['Josef Ressel']}",Buzola (také busola) je jednoduchý přístroj pr...,5571,Který český vynálezce sestrojil první buzolu?,Který český vynálezce sestrojil první buzolu?
3,"{'answer_start': [2084], 'text': ['1869']}","Deoxyribonukleová kyselina, běžně označovaná D...",7325,Ve kterém roce byla popsána deoxyribonukleová ...,Ve kterém roce byla popsána deoxyribonukleová ...
4,"{'answer_start': [168], 'text': ['strojové ins...","Centrální procesorová jednotka (zkratka CPU, a...",3770,Co vykonává procesor?,Co vykonává procesor?


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:00<00:00, 676kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 209k/209k [00:00<00:00, 422kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 112kB/s]


PreTrainedTokenizerFast(name_or_path='UWB-AIR/Czert-B-base-cased', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [16]:
print(tokenizer("Česko Slovensko má talent", "Masarykova Univerzita v Brne"))
print(tokenizer.decode([2, 5935, 9402, 2183, 12393, 3, 26054, 2116, 9457, 4166, 90, 18315, 1011, 3]))

{'input_ids': [2, 5935, 9402, 2183, 12393, 3, 26054, 2116, 9457, 4166, 90, 18315, 1011, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] Česko Slovensko má talent [SEP] Masarykova Univerzita v Brne [SEP]


In [18]:
# Max input sequence length is 512 so we set the max length to 384 and the maximum ovelap on sliding window to 128
max_length = 384
doc_stride = 128

random_example = df_train.iloc[4]
tokenized_example = tokenizer(
    random_example["question"],
    random_example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)
print("Length of tokenized question and context: ",len(tokenizer(random_example["question"],random_example["context"])["input_ids"]))
print("Maximum input length for current model: ", max_length)
print("Length of tokenized question and context with turnication: ",
      len(tokenizer(
          random_example["question"],
          random_example["context"],
          max_length = max_length,
          truncation = "only_second")
          ["input_ids"]))
print("Length of tokenized question and context with document stride: ",end="")
print([len(x) for x in tokenized_example["input_ids"]])

Length of tokenized question and context:  1667
Maximum input length for current model:  384
Length of tokenized question and context with turnication:  384
Length of tokenized question and context with document stride: [384, 384, 384, 384, 384, 384, 209]


In [57]:
#see the overlap with the stride
for i in tokenized_example["input_ids"]:
    print(tokenizer.decode(i))

[CLS] kolem kterého poledníku se rozkládá základní časové pásmo? [SEP] časové pásmo je ta část země, která používá stejný standardní čas. původně používali lidé sluneční čas, který má ovšem tu nevýhodu, že se liší od místa k místu. s rozvojem dopravy a komunikace byla tato nevýhoda stále výraznější, takže se postupem času přešlo na pásmový čas, kdy celá oblast země, zhruba 15 ° kolem daného poledníku, používá stejný čas, který je určen svým posunem od utc, koordinovaného světového času ( většinou je posun určen celistvým počtem hodin, jsou však i výjimky ). základním časovým pásmem je pásmo, ve kterém platí utc a které se rozkládá kolem nultého poledníku, který prochází královskou observatoří v greenwichi ( londýn, anglie ). z toho důvodu se pásmovému času odpovídajícímu utc někdy říká greenwichský čas ( gmt, greenwich mean time ). ostatní časová pásma jsou popsána rozdílem počtu hodin, o které se v nich platný čas liší od utc. např. středoevropský čas ( seč ) je označen jako utc + 1, 

To be able to create dataset for training we have to find starting and ending positions in every tokenized pair (question, context).
Since the dataset already dontains the starting positions of the answer in text we only need to add ending positions.
We can use `return_offsets_mapping` to be able to see span of every token from input IDs

In [19]:
random_example = df_train.iloc[6]
tokenized_example_offset = tokenizer(
    random_example["question"],
    random_example["context"],
    max_length=max_length,
    truncation="only_second",
    stride=doc_stride,
    return_overflowing_tokens = True,
    return_offsets_mapping = True
)
print(tokenized_example_offset["offset_mapping"][0][:20])
print(tokenizer.convert_ids_to_tokens(tokenizer(random_example["question"])["input_ids"][7]))
print(random_example["question"][tokenized_example_offset["offset_mapping"][0][7][0]:tokenized_example_offset["offset_mapping"][0][7][1]])

[(0, 0), (0, 4), (5, 7), (8, 12), (12, 15), (16, 24), (25, 30), (30, 33), (34, 42), (42, 43), (0, 0), (0, 8), (9, 14), (14, 17), (18, 26), (27, 28), (28, 36), (37, 42), (43, 49), (49, 50)]
##cké
cké


In [18]:
print(random_example["answers"]["answer_start"][0],random_example["answers"]["text"][0])
print(random_example["context"][:500])
print(tokenized_example_offset.sequence_ids().index(1))


1898 pro čavčuvenský
Korjačtina je jazyk z malé izolované čukotsko-kamčatské rodiny, kterým hovoří Korjaci, malý národ sídlící na ruském Dálném východě, především na Kamčatském poloostrově. Při sčítání lidu v roce 2002 uvedlo znalost korjackého jazyka 3 019 osob, tedy 34,5 % z celkového počtu Korjaků v Rusku. Procento Korjaků, kteří jazyk bez potíží ovládají, je však patrně mnohem nižší, podle některých odhadů jen 5,4 %. Z hlediska morfologické typologie se korjačtina řadí k aglutinačním jazykům s výskytem sufixů, p
16


In [20]:
print(tokenizer.decode(tokenized_example_offset.input_ids[0][:15]))

[CLS] pro který dialekt byla vytvořena první korjacká abeceda?


#### Function tokenizes row, splits into multiple parts if lenght > max_length and sets starting and ending positions of answer

In [20]:
def context_start_token_index(ex):
    return [x.index(1) for x in ex.token_type_ids]

def context_end_token_index(ex):
    return [(len(x) - x[::-1].index(1)) for x in ex.token_type_ids]

In [21]:
def get_tokenized_row(row, hf_dataset=True):
    tokenized_row = tokenizer(row["question"],
                              row["context"],
                              max_length=max_length,
                              truncation="only_second",
                              padding="max_length",
                              stride=doc_stride,
                              return_overflowing_tokens = True,
                              return_offsets_mapping = True)    
    tokenized_row["start_positions"] = []
    tokenized_row["end_positions"] = []
    context_starts = context_start_token_index(tokenized_row)
    context_ends = context_end_token_index(tokenized_row)
    index = tokenized_row["overflow_to_sample_mapping"]

    
    for i, o in enumerate(tokenized_row["offset_mapping"]):
        
        if hf_dataset:
            start_ch = row["answers"][index[i]]["answer_start"][0]
            end_ch = start_ch + len(row["answers"][index[i]]["text"][0])
        else:
            start_ch = row["answers"]["answer_start"][0]
            end_ch = start_ch + len(row["answers"]["text"][0])

        
        cls_ind = tokenized_row["input_ids"][i].index(tokenizer.cls_token_id)
        start = context_starts[i]
        end = context_ends[i] - 2
        if start_ch < o[start][0] or end_ch > o[end][1]:
            tokenized_row["start_positions"].append(cls_ind)
            tokenized_row["end_positions"].append(cls_ind)
        else:
            while o[start][0] < start_ch:
                start += 1
            while o[end][1] > end_ch:
                end -= 1
                if start > end:
                    end += 1
                    break
            tokenized_row["start_positions"].append(start)
            tokenized_row["end_positions"].append(end)
            
    return tokenized_row

In [22]:
print(df_train.shape[0])

9954


#### Delete questions with answers that contain tokens not recognised by models tokenizer

In [23]:
count = 0
data = []
for row in tqdm(df_train.iloc, total=df_train.shape[0], desc="Checking all answers and answers in contexts"):
    #print(row.question, row.answers["text"])
    test = get_tokenized_row(row, hf_dataset=False)
    answers = []
    #print(row.answers["text"])
    #print(test["start_positions"])
    #print(test["end_positions"])
    for i, z in enumerate(zip(test["start_positions"],test["end_positions"])):
        x=test["input_ids"][i][z[0]:z[1]+1]
        #print(tokenizer.decode(x))
        answers.append(tokenizer.decode(x))
    if row.answers["text"][0].replace(' ', '').lower() not in [x.replace(' ','').lower() for x in answers]:
        count += 1
        continue
    data.append([row.answers, row.question, row.context, row.id, row.title])
print("Deleted",count,"questions")
data_df = pd.DataFrame(data, columns=["answers", "question", "context", "id", "title"])

Checking all answers and answers in contexts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9954/9954 [01:43<00:00, 95.82it/s]

Deleted 134 questions


### Load SQAD to Huggingface dataset form pandas and split it into train and validation parts

In [ ]:
train_dataset = Dataset.from_pandas(data_df)
train_dataset = train_dataset.train_test_split(test_size=0.1)
train_dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'question', 'context', 'id', 'title'],
        num_rows: 8838
    })
    test: Dataset({
        features: ['answers', 'question', 'context', 'id', 'title'],
        num_rows: 982
    })
})

### Tokenize and preprocess the dataset

In [26]:
tokenized_train_dataset = train_dataset.map(get_tokenized_row, remove_columns=train_dataset["train"].column_names, batched=True)

Parameter 'function'=<function get_tokenized_row at 0x00000256B4E59AF0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:05<00:00, 65.37s/ba]


In [27]:
tokenized_train_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping', 'start_positions', 'end_positions'],
        num_rows: 112665
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping', 'start_positions', 'end_positions'],
        num_rows: 12207
    })
})

In [28]:
#Remove unused columns

tokenized_train_dataset["train"] = tokenized_train_dataset["train"].remove_columns("offset_mapping")
tokenized_train_dataset["test"] = tokenized_train_dataset["test"].remove_columns("offset_mapping")
tokenized_train_dataset["train"] = tokenized_train_dataset["train"].remove_columns("overflow_to_sample_mapping")
tokenized_train_dataset["test"] = tokenized_train_dataset["test"].remove_columns("overflow_to_sample_mapping")
tokenized_train_dataset["train"] = tokenized_train_dataset["train"].remove_columns("token_type_ids")
tokenized_train_dataset["test"] = tokenized_train_dataset["test"].remove_columns("token_type_ids")
tokenized_train_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 112665
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 12207
    })
})

## Training the model

In [30]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [36]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name, num_labels=2)
model.to(device)
model

Some weights of the model checkpoint at UWB-AIR/Czert-B-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from th

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [37]:
from transformers import TrainingArguments, Trainer, default_data_collator

In [38]:
training_arg = TrainingArguments(
                "BERT_czech_finetuned_sqad",
                evaluation_strategy = "epoch",
                learning_rate = 3e-5,
                num_train_epochs = 3,
                weight_decay = 0.1,
                per_device_train_batch_size = 16,
                per_device_eval_batch_size = 16,
                save_total_limit=2
                )

data_collator = default_data_collator

In [39]:
trainer = Trainer(
            model,
            training_arg,
            data_collator=data_collator,
            train_dataset=tokenized_train_dataset["train"],
            eval_dataset=tokenized_train_dataset["test"],
            tokenizer=tokenizer)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("BERT_czech_finetuned_sqad\\bert-czech-finetuned-sqad")